In [95]:
import git
import os
repo_url = 'https://github.com/sahajsoft/Pravarthan'
# clone the repo if it doesn't exist
expanded_path = os.path.expanduser('~/Pravarthan')

if not os.path.exists(expanded_path):
    repo = git.Repo.clone_from(repo_url, expanded_path)
else:
    repo = git.Repo(expanded_path)    


In [96]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.node_parser import SentenceSplitter
import chardet
import os

text = ""
accepted_file_extensions = [".py"]

def read_file_contents(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        if encoding:
            try:
                return raw_data.decode(encoding)
            except UnicodeDecodeError:
                print(f"UnicodeDecodeError: Unable to decode file '{file_path}' with detected encoding '{encoding}'.")
                return None
        else:
            print(f"Encoding detection failed for file '{file_path}'.")
            return None
        
for root, dirs, files in os.walk(expanded_path):
    for file_name in files:
        _, ext = os.path.splitext(file_name)
        if(ext in accepted_file_extensions):
            file_path = os.path.join(root, file_name)
            # Read and print contents of each file
            file_contents = read_file_contents(file_path)
            if file_contents is not None:            
                text+= file_contents
                text+= "\n"



Encoding detection failed for file '/Users/joey/Pravarthan/SahajIntranet/decorators.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/sales/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/sales/migrations/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/sales/serializers/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/deploy/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/migrations/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/tests/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/serializers/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/controllers/__init__.py'.
Encoding detection failed for file '/Users/joey/Pravarthan/projects/handlers/__init__.py'.
Encoding detection failed for

In [97]:
from llama_index.core.node_parser import CodeSplitter
from tree_sitter_languages import get_language, get_parser


# print(help(CodeSplitter))

nodes = []

node_parser = CodeSplitter(language="python", chunk_lines = 100, chunk_lines_overlap = 15, max_chars = 1500)
split_text = node_parser.split_text(text)
# print(dir(node_parser))
print(help(node_parser.get_nodes_from_documents))
nodes = node_parser.get_nodes_from_documents(
    [Document(text=text)], show_progress=True
)

Help on method get_nodes_from_documents in module llama_index.core.node_parser.interface:

get_nodes_from_documents(documents: Sequence[llama_index.core.schema.Document], show_progress: bool = False, **kwargs: Any) -> List[llama_index.core.schema.BaseNode] method of llama_index.core.node_parser.text.code.CodeSplitter instance
    Parse documents into nodes.

    Args:
        documents (Sequence[Document]): documents to parse
        show_progress (bool): whether to show progress bar

None


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [98]:
print(len(split_text))
print(split_text[0])
print(nodes[0])

2316
import os, django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "SahajIntranet.settings")
django.setup()

from openpyxl import load_workbook
import sys
from people.models import Member
from projects.models import Project
from timesheet.models import TimeCell, TimeCellRow

import pandas as pd
from datetime import datetime

time_row = {}
Node ID: a5acf240-db78-45f2-90e6-76daa8efeec5
Text: import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE",
"SahajIntranet.settings") django.setup()  from openpyxl import
load_workbook import sys from people.models import Member from
projects.models import Project from timesheet.models import TimeCell,
TimeCellRow  import pandas as pd from datetime import datetime
time_row = {}


In [99]:
import ollama
import weaviate
import weaviate.classes.config as wc
import weaviate.classes as wvc
import sys

embedding_model = "nomic-embed-text"

client = weaviate.connect_to_local()

if not client.collections.exists(name = 'test'):
    client.collections.create(name = 'test')

collection = client.collections.get('test')

# class_obj = {"class": "DocumentSearch", "vectorizer": "none"}
# client.schema.create_class(class_obj)

"""
A schema is defined, creating a custom class named “DocumentSearch”. This specific name doesn’t matter, 
but acts as an identifier for Weaviate, as you’ll see how we reference it later. 
The vectorizer is set to “none” since the vectorization is done externally using our embedding model.
"""

def gen_embeddings(value):
    try:
        response = ollama.embeddings(model=embedding_model, prompt=value)
    except Exception as e:
        print(f"failed to generate embedding for {value}: {e}")
    embedding = response["embedding"]
    return embedding

embeddings = []

for i, value in enumerate(split_text):
    embedding_i = wvc.data.DataObject(
        properties={
            'message': value,
        },
        vector=list(gen_embeddings(value))
    )
    embeddings.append(embedding_i)
    # Print the loading percentage
    print(f"\rLoading... {(i+1)*100/len(split_text)}%", end="")
    # Flush the output buffer to ensure the text is printed immediately
    sys.stdout.flush()



/var/folders/nd/6kk1vnrd6zjfdb3yzxzcchf00000gn/T/ipykernel_30779/2583587411.py:14: ResourceWarning: unclosed <socket.socket fd=76, family=30, type=1, proto=6, laddr=('::1', 59590, 0, 0), raddr=('::1', 8080, 0, 0)>
  collection = client.collections.get('test')


Loading... 100.0%82210708118%%%

In [100]:
print(client.is_ready())
print(client.is_connected())
print(dir(client))

print(help(collection.data.insert_many))
collection.data.insert_many(embeddings)


True
True
['_WeaviateClient__parse_connection_params_and_embedded_db', '_WeaviateClient__skip_init_checks', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_connection', 'backup', 'batch', 'close', 'cluster', 'collections', 'connect', 'get_meta', 'get_open_id_configuration', 'graphql_raw_query', 'integrations', 'is_connected', 'is_live', 'is_ready']
Help on method insert_many in module weaviate.collections.data:

insert_many(objects: Sequence[Union[~Properties, weaviate.collections.classes.data.DataObject[~Properties, Optional[Mapping[str, Union[str, uuid.UUID, Sequence[Union[str

BatchObjectReturn(all_responses=[UUID('636d83dc-1952-42a6-82c3-8abd3f7dcf66'), UUID('3bf44c3b-bc0a-4cea-a013-49c5ae7c2e4b'), UUID('77e7beec-8cd6-443e-9038-e880647033b7'), UUID('35fdbe1d-e840-4b15-a121-9005b226ae3f'), UUID('f7c41e16-3872-49dc-91db-dd25c7d334cd'), UUID('4b619991-2f80-4051-b25f-c6cfc9426a9e'), UUID('99b8d4b9-6d87-4d05-b15d-442c544135b0'), UUID('ff727d11-5ab0-40f5-ac69-c05a20c4b0fb'), UUID('ebf83305-be3b-434c-b97c-68acc8a682ba'), UUID('a6d7e272-2563-4195-91c1-699de2d12d38'), UUID('6b548ca7-d658-44ff-9467-1f9c80381d36'), UUID('ac9a7992-afd2-4e1d-afa9-fc3543da1d42'), UUID('b39f1985-5094-49ef-91f0-19b60c8bb3ed'), UUID('1182030c-d944-4b86-8a18-1ec256872be0'), UUID('f99b59a2-941e-478a-a016-b0df49361f68'), UUID('c9605bba-6ef7-4659-b048-074c4b15d795'), UUID('426fbf4a-6bfe-458a-ad8f-96ff74d63866'), UUID('a9fbb2bd-642d-431f-8f72-788d36922360'), UUID('290f02e9-cc2d-405d-a397-d96cdd4663cd'), UUID('96fd1806-a435-4c1b-8332-4b5b9982f09a'), UUID('4c727ea3-ed74-4e36-9ca4-57938e5c059d'), U

In [101]:
retrieved_docs = collection.query.near_vector(
    near_vector=gen_embeddings('''How can I get all active members'''),
    limit=15,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

retrieved_messages = [o.properties['message'] for o in retrieved_docs.objects]

[print(f"{i}\n\n") for i in retrieved_messages]

print(retrieved_messages[11])

l_of_all_the_active_members():
    active_emails = [member.work_email for member in Member.objects.all() if member.is_active()]
    return active_emails

from collections import defaultdict 
from people.models import Member
from people.pii_config import get_pii_host
from people.sadhak_migration.utils import get_members_present_in_new_entity
from people.services.member_service import call_sadhak_people_api
from people.serializers.member_serializer import MemberSerializer
import logging
import requests
import datetime

logger = logging.getLogger(__name__)

def get_all_members_insurance_data_for_sadhak():
    country_label_mapping = {"UK": 1, "AUS": 2}
    member_insurance_data = defaultdict(list)
    for member in get_members_present_in_new_entity():
        country_label = member.location.location.biz_unit.country.label
        all_dependant_details = member.insurance_details()
        key = country_label_mapping.get(country_label, 3)
        member_insurance_data[key].extend(structure_